In [1]:
# !pip install kafka-python

In [2]:
# !pip install tqdm

In [1]:
import pandas as pd
from kafka import KafkaProducer
from datetime import datetime
from json import dumps
from tqdm import tqdm

In [2]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))
if producer.bootstrap_connected():
    print(f"Successfully connected to bootstrap server")
else:
    print("Couldn't connect to bootstrap server.")

TOPIC_NAME = "ml-raw-dns"

Successfully connected to bootstrap server


In [3]:
def produce_message(producer_instance, topic, message):
    producer_instance.send(topic, message)
    producer_instance.flush()
    return

In [4]:
df = pd.read_csv("test.csv")

df.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')

df['label'] = df['label'].apply(lambda x: 0 if x == 'BENIGN' else 1)

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

df.to_csv("new_test.csv", index=False)

print("Dataset updated: dropped 'Unnamed: 0', converted labels, reshuffled rows.")
print("New shape:", df.shape)
print(df.head())

Dataset updated: dropped 'Unnamed: 0', converted labels, reshuffled rows.
New shape: (81503, 79)
   destination port  flow duration  total fwd packets  total backward packets  \
0               443       15106280                 21                      19   
1                53            153                  2                       2   
2             29080        6178179                  1                       6   
3                53            219                  2                       2   
4                53          25775                  1                       1   

   total length of fwd packets  total length of bwd packets  \
0                         9091                        16744   
1                           84                          140   
2                            6                           36   
3                           76                          154   
4                           49                          177   

   fwd packet length max  fwd packet 

In [5]:
with open("new_test.csv") as f:
    start_time = datetime.now()
    for i, line in tqdm(enumerate(f)):
        produce_message(producer_instance=producer, topic=TOPIC_NAME, message=line)
    end_time = datetime.now()
    print(f"Batch took {end_time-start_time} time for ingesting data")

print("Ingestion Completed")

81504it [01:41, 803.80it/s]

Batch took 0:01:41.419581 time for ingesting data
Ingestion Completed
